In [46]:
import pybgpstream
import pprint
import numpy as np

In [16]:
#collector = 'route-views.saopaulo'

#target_as = 18353#10318

def bgpstream(time_init, time_end, collector, target_as, ASPATHS, PREFIXES):
    stream = pybgpstream.BGPStream (
        from_time = time_init,
        until_time = time_end,
        filter = "type ribs and collector %s and path %s" % (
            collector,
            target_as
        )
    )

    #ASPATHS= set()
    #PREFIXES= set()

    for elem in stream:
        if int(elem.fields["as-path"].split(' ')[ -1]) == target_as:
            path = elem.fields["as-path"].split(' ')
            path_noprepend = []
            for i in range(0, len(path)):
                if i-1 > 0 and path[i] == path[i-1]:
                    continue
                path_noprepend.append(path[i])
                
            ASPATHS.add(tuple(path_noprepend))
            #ASPATHS.add(tuple(np.array(elem.fields["as-path"].split(' ')).astype(int).tolist()))
            PREFIXES.add(elem.fields["prefix"])       
            # pprint.pprint(elem.fields)

In [22]:
ASPATHS = set()
PREFIXES = set()
collector = 'route-views.saopaulo'

target_as = 18353
target_isp = 10318

time_init = "2012-03-01"
time_end = "2012-03-01 00:08"

bgpstream(time_init, time_end, collector, target_isp, ASPATHS, PREFIXES)

In [23]:
i = 0
for elem in ASPATHS:
    print(elem)
    i += 1
print("Caminos observados: ")
print(i)

('28138', '16735', '12956', '22927', '10481', '10318')
('28289', '53131', '10429', '12956', '22927', '10481', '10318')
('262757', '16735', '12956', '22927', '10481', '10318')
('16735', '3549', '10481', '10318')
('28289', '53131', '16735', '3549', '10481', '10318')
('262757', '16735', '3549', '10481', '10318')
('28138', '16735', '3549', '10481', '10318')
('16735', '12956', '22927', '10481', '10318')
Caminos observados: 
8


In [24]:
#Get providers

providers = set()

for path in ASPATHS:
    providers.add(path[-2])

In [25]:
providers

{'10481'}

In [26]:
#Get prefixes
i = 0

for prefix in PREFIXES:
    print(prefix)
    i += 1
    
print("Cantidad de prefijos: ", i)

201.235.0.0/19
181.164.32.0/19
190.244.128.0/19
186.136.96.0/19
186.137.192.0/19
200.114.128.0/19
201.235.128.0/19
190.19.224.0/19
24.232.96.0/19
200.49.128.0/19
200.114.160.0/19
181.164.128.0/19
190.19.64.0/19
190.17.160.0/19
186.136.0.0/19
190.245.192.0/19
190.244.32.0/19
190.244.192.0/19
201.235.32.0/19
190.17.32.0/19
190.16.96.0/19
24.232.192.0/19
186.137.0.0/19
200.49.128.0/20
186.136.64.0/19
190.245.64.0/19
181.164.0.0/19
190.247.96.0/19
201.235.224.0/19
190.16.192.0/19
200.114.224.0/19
201.231.0.0/19
186.136.0.0/15
181.164.96.0/19
190.246.224.0/19
190.17.64.0/19
190.19.96.0/19
190.244.0.0/14
200.114.128.0/17
190.18.32.0/19
190.18.224.0/19
190.247.192.0/19
190.18.64.0/19
201.235.64.0/19
186.136.0.0/14
190.246.0.0/19
200.126.224.0/19
190.17.192.0/19
201.231.64.0/19
190.17.0.0/19
24.232.0.0/16
200.89.128.0/19
190.16.0.0/19
190.16.224.0/19
190.19.0.0/19
24.232.160.0/19
190.247.0.0/19
190.247.224.0/19
201.231.192.0/19
190.247.32.0/19
201.231.128.0/19
201.235.160.0/19
190.247.128.0/19

Full RIB

In [47]:
RIB = set()

collector = 'route-views.saopaulo'

time_init = "2015-02-12 00:00:00"
time_end = "2015-02-12 00:00:00"

stream = pybgpstream.BGPStream (
    from_time = time_init,
    until_time = time_end,
    record_type="ribs",
    filter = "collector %s" % (
        collector
    )
)

for elem in stream:
    path = elem.fields["as-path"].split(' ')
    path_noprepend = []
    for i in range(0, len(path)):
        if i-1 > 0 and path[i] == path[i-1]:
            continue
        path_noprepend.append(path[i])
                        
    RIB.add((tuple(path_noprepend), elem.fields["prefix"]))

In [48]:
len(RIB)

380346

In [14]:
elem = RIB.pop()
elem

(('28329', '2914', '1299', '53755'), '23.226.64.0/20')

In [49]:
prefixes = {}

for elem in RIB:
    origin = elem[0][-1]
    if origin not in prefixes:
        prefixes[origin] = set()
    prefixes[origin].add(elem[1])

In [50]:
#Number of origins
len(prefixes)

3575

In [51]:
#Number of prefixes
i = 0
for origin in prefixes:
    i += len(origin)

print("Number of prefixes: ", i)

Number of prefixes:  17609


In [52]:
#Transform set into list
for origin in prefixes:
    prefixes[origin] = list(prefixes[origin])

In [ ]:
def same_possible_network(ip1_p, ip2_p):
    if not '/' in ip1_p or not '/' in ip2_p:
        return False
    
    ip1, prefix1 = ip1_p.split('/')
    ip2, prefix2 = ip2_p.split('/')
    
    num_prefix1 = int(prefix1)
    num_prefix2 = int(prefix2)
    
    #If they have the same prefix, discount 1 to both to check
    #if they are in the same network, so that i can aggregate them
    if (num_prefix1 == num_prefix2):
        num_prefix1 -= 1
        num_prefix2 -= 1
    
    bits_for_contained = min(num_prefix1, num_prefix2)
    
    numbers1 = list(map(int, ip1.split('.')))
    numbers2 = list(map(int, ip2.split('.')))
    
    size = int(bits_for_contained / 8)
    last_bits = bits_for_contained % 8

    status = True
    
    for i in range(0, size):
        if (numbers1[i] != numbers2[i]):
            status = False
            break;

    if (last_bits and (numbers1[size] & (256-pow(2, (8-last_bits)))) != (numbers2[size] & (256-pow(2, (8-last_bits))))):
        status = False

    return status
    
            
def perform_agg(ip1_p, ip2_p):
    ip1, prefix1 = ip1_p.split('/')
    ip2, prefix2 = ip2_p.split('/')
    
    #If they have the same prefix, i have to aggregate
    #and discount 1 to the prefix starting which ip is "less", example:
    # 40.165.32.0/20 and 40.165.48.0/20
    # that is 40.165.32.0/19
    if (int(prefix1) == int(prefix2)):
        size = int((32-int(prefix1)-1) / 8)
        
        numbers1 = list(map(int, ip1.split('.')))
        numbers2 = list(map(int, ip2.split('.')))
        
        if (numbers1[size] < numbers2[size]):
            return ip1 + '/' + str(int(prefix1)-1)
        
        return ip2 + '/' + str(int(prefix1)-1)
    
    #If they have different prefix, return the one with the minimum prefix
    
    if (int(prefix1) < int(prefix2)):
        return ip1
    
    return ip2
            
#Continous and aggregates if possible
def contiguous_and_aggregates(ip, prefixes):
    for ip2 in prefixes:
        #Skip same ip
        if ip == ip2:
            continue
        
        if (not same_possible_network(ip, ip2)):
            continue
            
        new_ip = perform_agg(ip, ip2)
        prefixes.remove(ip)
        prefixes.remove(ip2)
        prefixes.append(new_ip)
        return True
    
    return False

# Maximum aggregation    
def perform_max_agg(prefixes):
    #Try ips until some aggregation is done
    for ip in prefixes:
        if (contiguous_and_aggregates(ip, prefixes)):
            perform_max_agg(prefixes)
            break;
            
for origin in prefixes:
    perform_max_agg(prefixes[origin])

In [75]:
#Number of prefixes
i = 0
for origin in prefixes:
    i += len(origin)

print("Number of prefixes: ", i)

Number of prefixes:  17609
